In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import albumentations as A
from tqdm import tqdm
import copy
import ctypes

import matplotlib.pyplot as plt
import seaborn as sns


import cv2
from PIL import Image
import io
import h5py
import os, gc, time

# Deep learning library
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms # use pytorch data argument 
from torchvision.models import efficientnet_b0, efficientnet_v2_m, swin_b
from torchvision import models

# Accelerate parts
from accelerate import Accelerator, notebook_launcher # main interface, distributed launcher
from accelerate.utils import set_seed # reproducability across devices



# Sklearn train test split/ LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_curve, auc ,roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay


#system
import psutil

import warnings
warnings.filterwarnings("ignore")



In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def memoryInfo():
    memory_info = psutil.virtual_memory()

    # Print the used and available memory
    print(f"Total Memory: {memory_info.total / (1024 ** 3):.2f} GB")
    print(f"Available Memory: {memory_info.available / (1024 ** 3):.2f} GB")
    print(f"Used Memory: {memory_info.used / (1024 ** 3):.2f} GB")
    print(f"Percentage Used: {memory_info.percent}%")
    

In [ ]:
memoryInfo()

In [ ]:
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        gc.collect()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        time.sleep(0.3)

In [ ]:
clearMemory()

In [ ]:
!pip show accelerate # build-in support accelerate for multiple-GPU , later implement pytorch multiple GPU

## From Timm Library find support CNN/transformer model name
#### [Timm Docs](https://timm.fast.ai/)

In [ ]:
avail_pretrained_models = timm.list_models(pretrained=False)
len(avail_pretrained_models), avail_pretrained_models[:5]

In [ ]:
avail_pretrained_models[250:400]

In [ ]:
timm.list_models("*efficientnet*", pretrained=True)

In [ ]:
timm.list_models('*swin*', pretrained=True)

In [ ]:
timm.list_models('*vit*', pretrained=True)

# Define Configure and Path

In [ ]:
class CFG:
    seed = 42
    test_size= 0.2 # for evaluation
    
    # dataset 
    imgSize =  300 #312 # 224  # avoid use large image size (memory size limitation)
    img_resize=True # False if pad with zeros instead of resize
    
    # define used CNN/transformer Model name from timm pytorch image 
    modelName1 = "efficientnet_b0.ra_in1k" #  efficientnet v1 b0
    modelName2 = "tf_efficientnetv2_b1.in1k" # efficientnet v2 b1
    modelName3 = "swin_tiny_patch4_window7_224.ms_in22k" # swin transformer v1 base
    modelName4 = "swinv2_cr_tiny_ns_224.sw_in1k" # swin transformer v2 tiny
    modelName5 =  "tf_efficientnet_b1.ns_jft_in1k" #efficientnet v1 b1
    modelName6 = "tf_efficientnetv2_b2.in1k" # efficientnet v2 b2  #avoid use b2 model easy GPU out of memory 

    # train model parameter
    pretrained = False #define not use pretrained weight
    lastLayerHiddenDim = 1280
    outDim = 1

    lr = 2e-4  # r
    weightDecay = 1e-5
    trainBatchSize = 64 #32#64
    valBatchSize =  64 #32#64#128
    inferBatchSize = 64 #32#64#128
    trainEpochs = 50 #1#2#3
    numClass = 2
    
    USE_EFFECTNetv1 = True#False#True
    USE_EFFECTNetv2 = True
    USE_SWINTransV1 = False#True#False
    USE_SWINTransV2 = False
    modelSaveDir = "/kaggle/working/"
    
    # pAUC core paramer
    tprTh = 0.8
    
    train = True # set train 
    infer = True # set test 
    
class PATHS:
    trainImagePath = "/kaggle/input/isic-2024-challenge/train-image.hdf5"
    testImagePath = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
    trainMetaDataPath= "/kaggle/input/isic-2024-challenge/train-metadata.csv"
    testMetaDataPath = "/kaggle/input/isic-2024-challenge/test-metadata.csv"
    submissionPath = "/kaggle/input/isic-2024-challenge/sample_submission.csv"
    
    

In [ ]:
def seed_everything(seed):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(CFG.seed)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.is_available():
#     accelerator = Accelerator()
#     device = accelerator.device
# else:  
#     device = torch.device("cpu")

device

In [ ]:
if torch.cuda.is_available():
    !nvidia-smi

# Load Dataset

In [ ]:
trainMetaData = pd.read_csv(PATHS.trainMetaDataPath)
testMetaData = pd.read_csv(PATHS.testMetaDataPath)
submission = pd.read_csv(PATHS.submissionPath)

In [ ]:
trainMetaData

In [ ]:
testMetaData

In [ ]:
submission

# EDA

In [ ]:
def printUniqueValue(df, showAll= True):
    for col in df.columns:
        if showAll ==True:
            print(f"""{col} :  {df[col].unique()}""")
        else:
            if df[col].dtype == "object": # only show object type columns unique values
                print(f"{col} : {df[col].unique()}")

In [ ]:
printUniqueValue(trainMetaData, True)

In [ ]:
trainMetaData["target"].value_counts()

### Seem Class is extremely imbalance

In [ ]:
# seem the dataset a lot of null value
trainMetaData.isnull().sum()

In [ ]:
trainMetaData["iddx_full"].value_counts()

In [ ]:
trainMetaData["sex"].hist()

In [ ]:
trainMetaData.age_approx.hist();

In [ ]:
trainMetaData.anatom_site_general.hist();

In [ ]:
trainMetaData.patient_id.value_counts()

### show level of confident of train dataset

In [ ]:
trainMetaData.tbp_lv_dnn_lesion_confidence.hist(); 

In [ ]:
# find target == 1 confident level distribution
trainMetaData[trainMetaData["target"] == 1].tbp_lv_dnn_lesion_confidence.hist();

## Extract Image dataset (train/test)

In [ ]:
def readImghdf5(filePath, transform=False):
    """
    avoid use this function, image arary data alway load in memory ,easy has out of memory issue
    """
    with h5py.File(filePath, 'r') as f:
        idsList = list(f.keys())
        print("Number of image files: ", len(idsList))
        idsImg = {}
        for i, imgId in enumerate(tqdm(idsList)):
            imgRaw = f[imgId][()]
            img = Image.open(io.BytesIO(imgRaw))
            idsImg[imgId] = np.array(img, dtype=np.uint8) #store in idsImg
            
            # Clear the image variable to save memory
            del img
            del imgRaw
            if i % 1000 == 999:
                gc.collect()
        if transform: #gnerate transform image
            pass
    
#     gc.collect()
    return idsImg

In [ ]:
clearMemory()

In [ ]:
memoryInfo()

In [ ]:
%%time
# load data
# imageTrain = readImghdf5(PATHS.trainImagePath) # convert all image store in memory ( too large memory consume)
# imageTest = readImghdf5(PATHS.testImagePath)  # convert all image store in memory (too large memory consume)
trainHDF5 = h5py.File(PATHS.trainImagePath, 'r')
testHDF5 = h5py.File(PATHS.testImagePath, 'r')

#### Check image size distribution

In [ ]:
clearMemory()

# data Clearning 

In [ ]:
trainMetaData["lesion_id"].notnull().sum() # check how many valid data

In [ ]:
trainMetaData["lesion_id"].isnull().sum() # check how many Null invalid data

In [ ]:
trainMetaData["lesion_id"]

In [ ]:
# only filter the valid lesion id dataset
trainMetaData = trainMetaData[trainMetaData["lesion_id"].notnull()]
trainMetaData

In [ ]:
trainMetaData["lesion_id"].isnull().sum() #check null 

# Seem train dataset extremely imbalance

In [ ]:
trainMetaData["target"].value_counts()

In [ ]:
393/ (21665+393) * 100 # cancer image around 1.78% data

In [ ]:
trainMetaDataHasCancer= trainMetaData[trainMetaData["target"] == 1]
trainMetaDataHasCancer.shape

In [ ]:
trainMetaDataNoCancer= trainMetaData[trainMetaData["target"] == 0]
trainMetaDataNoCancer

In [ ]:
# trainMetaDataHasCancer.info()

### Because 393 true cancar train sample data size is not enough to effective train Deep learning model, need generate more than 5 to 20 times true data by data Augmentation technique

In [ ]:
# repeat row data n Time
numGenImage = 10#5
newTrainMetaDataHasCancer = pd.DataFrame(np.repeat(trainMetaDataHasCancer.values, numGenImage, axis=0),
                                            columns=trainMetaDataHasCancer.columns)
# newTrainMetaDataHasCancer = trainMetaDataHasCancer
newTrainMetaDataHasCancer.shape

In [ ]:
newTrainMetaDataHasCancer["isic_id"].value_counts()

In [ ]:
newTrainMetaDataHasCancer.info()

In [ ]:
# convert target dtype in int64
newTrainMetaDataHasCancer["target"] = newTrainMetaDataHasCancer["target"].astype(np.int64)

In [ ]:
newTrainMetaDataHasCancer.info()

In [ ]:
len(newTrainMetaDataHasCancer)

In [ ]:
# downsample for no cancer training data
# trainMetaDataNoCancer = trainMetaDataNoCancer.sample(n=len(trainMetaDataHasCancer), random_state=42)
trainMetaDataNoCancer = trainMetaDataNoCancer.sample(n=len(newTrainMetaDataHasCancer), random_state=42)
trainMetaDataNoCancer.shape

In [ ]:
# balancetrainMetaData = pd.concat([trainMetaDataHasCancer, trainMetaDataNoCancer], axis=0) # combine two dataset in row axis
balancetrainMetaData = pd.concat([newTrainMetaDataHasCancer, trainMetaDataNoCancer], axis=0) # combine two dataset in row axis
balancetrainMetaData.shape

In [ ]:
# random sample order of balanced train dataset
balancetrainMetaData = balancetrainMetaData.sample(frac=1).reset_index(drop=True)
balancetrainMetaData

In [ ]:
balancetrainMetaData.target.value_counts().plot(kind="bar");

In [ ]:
del trainMetaData

## Create Custom DataSet

In [ ]:
class SKINCancerDatset(Dataset):
    def __init__(self, 
                 metaData: pd.DataFrame,  # metadata
                 idsImg: dict, # image
                 resize : bool=True,
                 train : bool=True):
        self.metaData = metaData
        self.idsImg = idsImg
        self.resize = resize
        self.train = train # indicate it is a training dataset, with labels
    
    def __len__(self):
        return len(self.metaData)
    
    def padImg(self, img):
        padX = CFG.imgSize - img.shape[1] # calulate pad X size
        padY = CFG.imgSize - img.shape[0] # calulate pad Y size
        paddedImg =  np.pad(img, 
                            ((padY//2, padY//2 + padY%2),
                             (padX//2, padX//2 + padX%2),
                             (0, 0)),
                            mode='constant', constant_values=0)
        return paddedImg
        
    def __getitem__(self, item):
        row= self.metaData.iloc[item] # get row 
        isicId = row.isic_id
        img = self.idsImg[isicId]  # normalize
        if self.resize:
            img = cv2.resize(img, dsize=(CFG.imgSize, CFG.imgSize))
        else: # use padding image
            img = self.padImg(img)
        if self.train: # is training dataset
            label = row.target
            return img, label
        else: # test dataset (no target)
            return img

In [ ]:
class SKINCancerDatsetV2(Dataset):
    def __init__(self, 
                 metaData ,  # metadata
                 hdf5File , # image file 
                 transform , # transform function 
                 train : bool=True):
        self.metaData = metaData
        self.hdf5File = hdf5File
        self.transform = transform
        self.train = train # indicate it is a training dataset, with labels
    
    def __len__(self):
        return len(self.metaData)
        
        
    def __getitem__(self, item):
        row= self.metaData.iloc[item] # get row 
        isicId = row.isic_id
        img =  Image.open(io.BytesIO(self.hdf5File[isicId][()])) # convert byte array to PIL image#self.idsImg[isicId]  # normalize
        if self.transform: 
            img = self.transform(img)#cv2.resize(img, dsize=(CFG.imgSize, CFG.imgSize))
            
        if self.train: # is training dataset
            label = row.target
            return img, label
        else: # test dataset (no target)
            return img

## Define transform function 

In [ ]:
valtransformFuct = transforms.Compose([
        transforms.Resize((CFG.imgSize, CFG.imgSize)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalize
        ])

traintransformFuct = transforms.Compose([
        transforms.Resize((CFG.imgSize, CFG.imgSize)),
        transforms.RandomRotation(degrees=(0, 150)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
#         transforms.RandomCrop((CFG.imgSize, CFG.imgSize))
#         transforms.ColorJitter(brightness=(0.8,1.2), 
#                                contrast=(1)),
#                                saturation=(0.6,1.5)),
#                                hue=(-0.5, 0.5)),
        transforms.RandomAdjustSharpness(sharpness_factor=2.0, p=0.5),
        transforms.RandomInvert(p=0.2),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalize
        ])

# Split Train/test dataset

In [ ]:
if CFG.train:
#     train, val = train_test_split(trainMetaData,
#                                   stratify= trainMetaData.target,
#                                   test_size= CFG.test_size,
#                                   random_state= CFG.seed)
    train, val = train_test_split(balancetrainMetaData,
                                  stratify= balancetrainMetaData.target,
                                  test_size= CFG.test_size,
                                  random_state= CFG.seed)
    print(train.target.value_counts(), val.target.value_counts())

## create train/valid dataset

In [ ]:
if CFG.train:
#     trainDataset = SKINCancerDatset(train, imageTrain,  resize=CFG.img_resize, train=True)
#     valDataset = SKINCancerDatset(val, imageTrain, resize=CFG.img_resize, train=True)
    trainDataset = SKINCancerDatsetV2(train, trainHDF5, traintransformFuct, train=True)
    valDataset = SKINCancerDatsetV2(val, trainHDF5, traintransformFuct, train=True)
    # create train/valid data loader
    trainDataLoader =torch.utils.data.DataLoader(
        trainDataset,
        batch_size=CFG.trainBatchSize,
        num_workers=4,
        shuffle=True,
        pin_memory=True,
    )
    valDataLoader =torch.utils.data.DataLoader(
        valDataset,
        batch_size=CFG.valBatchSize,
        num_workers=4,
        shuffle=False,
        pin_memory=True,
    )

# test/submit dataset/dataloader
# testDataset = SKINCancerDatset(testMetaData, imageTest,  resize=CFG.img_resize, train=False)
testDataset = SKINCancerDatsetV2(testMetaData, testHDF5, valtransformFuct, train=False)
testDataLoader = torch.utils.data.DataLoader(
    testDataset,
    batch_size=CFG.inferBatchSize,
    num_workers=4,
    shuffle=False,
    pin_memory=True,
)

In [ ]:
len(testDataset)

In [ ]:
len(trainDataLoader)

In [ ]:
train["isic_id"].iloc[0]

In [ ]:
#check Train Dataloader for each batch
for x, y in trainDataLoader:
    print(f"Shape of X : {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    print(x[0].shape)
    plt.figure(figsize=(16, 8))
#     for i in range(16):
    for i in range(36):
        plt.subplot(6, 6, i+1)
        plt.imshow(x[i].permute(1, 2, 0))
        plt.title(train["isic_id"].iloc[i])  # This assumes you have direct access to IDs in this scope
        plt.axis('off')
    plt.show()
    break


In [ ]:
for x, y in valDataLoader:
    print(f"Shape of X : {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    print(x[0].shape)
    plt.figure(figsize=(16, 8))
    for i in range(36):
        plt.subplot(6, 6, i+1)
        plt.imshow(x[i].permute(1, 2, 0))
        plt.title(val["isic_id"].iloc[i])  # This assumes you have direct access to IDs in this scope
        plt.axis('off')
    plt.show()
    break

In [ ]:
for x in testDataLoader:
    print(f"Shape of X : {x.shape}")
#     print(f"Shape of y: {y.shape} {y.dtype}")
    plt.figure(figsize=(16, 8))
    for i in range(3):
        plt.subplot(4, 4, i+1)
        plt.imshow(x[i].permute(1, 2, 0))
        plt.title(testMetaData["isic_id"].iloc[i])  # This assumes you have direct access to IDs in this scope
        plt.axis('off')
    plt.show()
    break

In [ ]:
# plt.imshow(testDataset[0]);

# Reduce memory usage

In [ ]:
# delete unused reduce memory
# del imageTrain
# del imageTest
# del trainMetaData
# del train
# #del val

In [ ]:
# trainMetaData

In [ ]:
clearMemory()

# Create training / validation function for Pytorch

In [ ]:
def trainFunc(model, loader, optimizer, lossFn):
    """ one epoch training"""
    model.train() # set the model to training mode calculate gradients and update weights
#     size = len(loader) # total data size
    if device.type == "cuda":
        scaler = torch.cuda.amp.GradScaler(enabled=True)
#     else:
#         scaler = torch.cpu.amp.GradScaler(enabled=True)
    losses = []
    correct =0
    total= 0
    for i, data  in enumerate(tqdm(loader)):
#     for i, data in enumerate(loader, 0):
        inputs , labels = data # get into
        inputs  = inputs.to(device).float()
        labels = labels.to(device).view(-1, 1).float() # to GPU if available
#         inputs  = inputs.float()
#         labels = labels.view(-1, 1).float()# to GPU if available
        

        yhat = model(inputs)# prediction
#         print(inputs.shape)
#         print("yhat shape: ", yhat.shape)
        loss = lossFn(yhat, labels) # calculate loss
        losses.append(loss.item()) # record loss value  
        
        #predict
#         predicted = yhat.sigmoid().detach().cpu().numpy() # convert to numpy 
        predicted = torch.round(yhat.data) # convert into 0 or 1 (binary classifier)
#         print("predicted: ", predicted)
#         print("Labels: ", labels)
        total += labels.size(0)
#         correct += torch.sum(yhat == labels.data)#(predicted == labels).sum().item()
        correct += (predicted == labels).sum()
        
#         print("correct: ",correct)
        # reset previous gradient 
        optimizer.zero_grad()
        if device.type == "cuda":
#             accelerator.backward(loss)
            scaler.scale(loss).backward() #getting gradients
            scaler.step(optimizer) # update weight
#             optimizer.step()
            scaler.update()
        else:
            loss.backward() # calculate new gradient 
            optimizer.step() # update weight/parameter
            
        if i % 100 == 99: #each 100 mini-batches print the loss
            tempLoss = np.mean(losses)
            print(f"Batch {i+1},  Training Loss : {tempLoss}")
#             print("Total Correct: ", correct)
            tempAcc = 100 * correct.detach().cpu().numpy()/ (total)
            print(f"Batch {i+1}, Training Acc : {tempAcc} %")
            
        
        
    
    avgLoss = np.mean(losses)  #average losses
    accTrain  = 100 * correct.detach().cpu().numpy() /total
    print(f"Training Avg Loss: {avgLoss}")
    print(f"Training Accuracy: {accTrain}")
    return avgLoss , accTrain
        
    
    

In [ ]:
def validationFunc(model, loader, lossFn):
    model.eval() # set model to evaluation mode, no calcuate gradients
    losses = []
    preds = []
    correct =0
    total= 0
    with torch.no_grad():
        for i, data  in enumerate(tqdm(loader)):
#         for i , data in enumerate(loader, 0):
            inputs , labels = data # get x and y 
            inputs = inputs.to(device).float() 
            labels = labels.to(device).view(-1, 1).float() # to GPU if available
            
            #prediction 
            yhat = model(inputs)
            loss = lossFn(yhat, labels)
            predicted = torch.round(yhat.data) # convert into 0 or 1 (binary classifier)
            preds.append(predicted.detach().cpu().numpy()) # prediction
            losses.append(loss.item())
            
            #predict
#             predicted = yhat.sigmoid().detach().cpu().numpy()
            total += labels.size(0)
#             correct += torch.sum(yhat == labels.data)# (predicted == labels).sum().item()
            correct +=   (predicted == labels).sum()
            
            if i % 100 == 99: #each 100 mini-batches print the loss
                tempLoss = np.mean(losses)
                print(f"Batch {i+1},  Val Loss : {tempLoss}")
                tempAcc =  100 * correct.detach().cpu().numpy()/ (total)
                print(f"Batch {i+1}, Val Acc : {tempAcc} %")
    
    preds = np.concatenate(preds, 0)
    avgLoss = np.mean(losses) # 
    accVal  = 100 * (correct.detach().cpu().numpy()) /total
    print(f"Val Avg Loss: {avgLoss}")
    print(f"Val Accuracy: {accVal} %")
    return avgLoss, accVal , preds
    

# inference for submission

In [ ]:

def inferFunc(model, loader):
    model.eval() # set model to evaluation mode, no calcuate gradients
    preds = []
    with torch.no_grad():
        for i , data in enumerate(loader, 0):
            inputs = data.to(device).float()
            
            #preiction
            yhat = model(inputs)
            preds.append(yhat.sigmoid().detach().cpu().numpy())
    preds = np.concatenate(preds, 0)
    return preds

# Partial AUC socre function for metrics

In [ ]:
def plotPartialAUC(fpr, tpr, auc):
    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc)
    plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for  Classification')
    plt.legend()
    plt.show()

In [ ]:
def computePAUC(ytrue, ypred, tprThreshold=0.8):
    """
    computer Partial AUC score with TPR Threshold
    """
    # computer ROC curve
    fpr, tpr, thresholds = roc_curve(ytrue, ypred)
    
    # Find the indices where the TPR is above the threshold
    tprAboveThr = np.where(tpr>= tprThreshold)[0]
    
    if len(tprAboveThr) ==0:
        return 0.0 #  all below tpr threshold
    
    #Extract index for ROC segment about threshold
    start = tprAboveThr[0]
    fprAboveThr = fpr[start:]
    tprAboveThr = tpr[start:] - tprThreshold
    
    partialAUC = auc(fprAboveThr, tprAboveThr)
#     plotPartialAUC(fprAboveThr, tprAboveThr, partialAUC)
    
    return partialAUC
    
    
    

In [ ]:
def computePAUCV2(targets, predictions, tprThreshold=0.80):
    # Ensure the inputs are numpy arrays for processing
    targets = np.array(targets)
    predictions = np.array(predictions)

    v_gt = abs(targets - 1)  # Assuming 'targets' are 0s and 1s
    v_pred = 1.0 - predictions  # Inverting predictions if necessary

    max_fpr = abs(1 - tprThreshold)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

    # Adjust scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

# Test Computing pAUC

In [ ]:

y_true = np.array([0, 1, 0])
y_preds = np.array([0.1, 0.4, 0.35])

pauc = computePAUC(y_true, y_preds)
print(f'Partial AUC above 80% TPR: {pauc:.4f}')

# Training Val Loop function

In [ ]:
trainLossHist = []
trainAccHist = []
valLossHist =[]
valAccHist = []
paucHist = []
epochList = []
modelHistory= {}

def clearTrainHist():
    global trainLossHist,  trainAccHist, valLossHist,  valAccHist, \
            epochList, bestLoss, paucHist
    
    trainLossHist.clear()
    trainAccHist.clear()
    paucHist.clear()
    valLossHist.clear()
    valAccHist.clear()
    epochList.clear()


In [ ]:
# TrainLoop function
def trainValLoop(model, optimizer, modelName, epochs=50):
    global trainLossHist, trainAccHist, paucHist, \
            valLossHist, valAccHist, epochList, modelHistory, bestAUC, lossFn
    
    #Initialize Variables for EarlyStopping
    best_model_weights = None
    patience = 10 #5#20#15
    bestScore= 0 # use pAUP
    bestAcc =0
    bestLoss = np.inf   # init to infinity

    for i in range(epochs):
        print(f"Epochs: {i+1}")
        print("Training:")
        trainLoss , trainAcc = trainFunc(model, trainDataLoader, optimizer, lossFn)
        trainLossHist.append(trainLoss)
        trainAccHist.append(trainAcc)
        print("Validation:")
        valLoss,  valAcc,  preds= validationFunc(model, valDataLoader, lossFn)
#         print("All preds value: ", preds)
#         print("length of preds : ", len(preds))
#         print("type of preds :" , type(preds))
#         score = computePAUC(val.target.values, 
#                             preds,
#                             tprThreshold=CFG.tprTh)
        score = computePAUCV2(val.target.values, 
                            preds,
                            tprThreshold=CFG.tprTh)
        print(f"pAUC Score: {score}")
        valLossHist.append(valLoss)
        valAccHist.append(valAcc)
        paucHist.append(score)
        epochList.append(i+1)
        
        if i % 3 == 2:
            cm = confusion_matrix(val.target.values, preds)
            print(f"\n\rClassification Report For {modelName} :\n\r", classification_report(val.target.values, preds))
            cmd =ConfusionMatrixDisplay(cm, display_labels=[False, True])
            cmd.plot()
            plt.title(f"Confusion Matrix for {modelName}")
            plt.show()

        
        # update bestloss, best accuracy , best score
        if valLoss < bestLoss:
            bestLoss = valLoss
            
        if valAcc > bestAcc:
            bestAcc = valAcc
            
        if score > bestScore:
            bestScore = score
            best_model_weights = copy.deepcopy(model.state_dict()) #  copyu the best model
            patience  = 10 #5 #20# 15  # reset counter
            torch.save({"model": model.state_dict(),
                         'predictions': preds
                       }, (modelName + ".pt"))
        else:
            patience -= 1
            if patience == 0:
                print("Early Stop!")
                break
                
        gc.collect()
    
    # Load the best model weights
    model.load_state_dict(best_model_weights)
    
    # append Model History for comparsion 
    modelHistory[modelName] = {
        "train_loss": trainLossHist,
        "train_acc": trainAccHist,
        "val_loss": valLossHist,
        "val_acc" : valAccHist,
        "pauc": paucHist,
        'epoch_list': epochList,
        "best_score": bestScore,
        "best_acc": bestAcc,
        "best_loss": bestLoss
    }
    
                

In [ ]:
clearMemory()

## Test direct model call

In [ ]:
# # model = timm.create_model('tf_efficientnetv2_s.in21k', pretrained=True)
# model = timm.create_model("efficientnet_b3.ra2_in1k", pretrained=False)
# model
# # del model

In [ ]:
# num_features = model.classifier.in_features  # Grab the number of input features to the classifier
# num_featuresL2 = num_features //2
# model.classifier = nn.Sequential(
#     nn.Dropout(0.2),
#     nn.Linear(num_features, num_featuresL2),
#     nn.Linear(num_featuresL2, 1),
#     nn.Sigmoid()
# )

In [ ]:
# model.classifier

In [ ]:
# del model

# CNN/Vision Transformer Approach Model

In [ ]:
class EfficientModel(nn.Module):
    def __init__(self, config , modelName, pretrained= False, dropOut=0.0):
        super(EfficientModel, self).__init__()
        self.backbone = timm.create_model(model_name=modelName, pretrained=pretrained)
        self.numFeatureL1 = self.backbone.classifier.in_features  # Grab the number of input features to the classifier
        self.numFeatureL2 = self.numFeatureL1 //2
        # modify classifier (final layer for custom layer)
#         self.backbone.classifier = nn.Sequential(
#                     nn.Dropout(dropOut),
# #                     nn.Linear(self.numFeatureL1, self.numFeatureL2),
# #                     nn.Linear(self.numFeatureL2, 1),
#                     nn.Linear(self.numFeatureL1, 1),
#                     nn.Sigmoid()
#         )
        self.backbone.classifier = nn.Sequential(
#                     nn.Dropout(dropOut),
                    nn.Linear(self.numFeatureL1, self.numFeatureL2),
                    nn.ReLU(),
                    nn.Dropout(dropOut),
                    nn.Linear(self.numFeatureL2, 1),
#                     nn.Linear(self.numFeatureL1, 1),
                    nn.Sigmoid()
        )
        
    def forward(self, x):
#         print("X input shape before reorder: ", x.shape) # check input shape
#         inputs = x.permute(0, 3, 1, 2) # change data order format feed into model (batch, channel, H, W) for dataset v1
        inputs = x  # for dataset V2 transformed to tensor not need reorder the data shape
#         print("X inputs shape After reorder: ", inputs.shape) # check reordered shape
        out = self.backbone(inputs)
        return out
        

### for SWIN transformer

In [ ]:
class SWINTransModel(nn.Module):
    def __init__(self, config , modelName, pretrained= False, dropOut= 0.0):
        super(SWINTransModel, self).__init__()
        self.backbone = timm.create_model(model_name=modelName, pretrained=pretrained)
        self.numFeatureL1 = self.backbone.head.in_features  # Grab the number of input features to the classifier
        self.numFeatureL2 = self.numFeatureL1 //2
        # modify head(final layer for custom layer)
#         self.backbone.head = nn.Sequential(
#                     nn.Dropout(dropOut),
# #                     nn.Linear(self.numFeatureL1, self.numFeatureL2),
# #                     nn.Linear(self.numFeatureL2, 1),
#                     nn.Linear(self.numFeatureL1, 1),
#                     nn.Sigmoid()
#         )
        self.backbone.head.fc.out_features= 1
        
    def forward(self, x):
            print("X input shape before reorder: ", x.shape) # check input shape
#             inputs = x.permute(0, 3, 1, 2) # change data order format feed into model
            inputs = x  # for dataset V2 transformed to tensor not need reorder the data shape
            out = self.backbone(inputs) 
            out = F.sigmoid(out)
            return out
        

In [ ]:
# modeltest = timm.create_model(CFG.modelName3, pretrained=False)
# modeltest.head.fc.out_features =1
# modeltest
# # # del modelf

# inital Model

In [ ]:
if CFG.USE_EFFECTNetv1:
    selectModel = CFG.modelName5 #CFG.modelName1
    efficientNet1 = EfficientModel(CFG, selectModel, pretrained=False, dropOut=0.1)
    efficientNet1.to(device)
    optimizer1 = torch.optim.Adam(efficientNet1.parameters(), lr=2e-4)
    print(efficientNet1)

## direct test 

In [ ]:
# efficientNet1.eval() 

# # Generate a random tensor with the size matching the input size of the model
# # Assuming the input size 384x384 for this example
# input_tensor = torch.randn(1, 3, 384, 384)  # 1 is the batch size

# # Forward the random tensor through the model
# with torch.no_grad():  # Turn off gradients for forward pass
#     output = efficientNet1(input_tensor)

# # Print the output
# print(output)

In [ ]:
if CFG.USE_EFFECTNetv2:
    selectModel = CFG.modelName2 #CFG.modelName6
    efficientNetV2 = EfficientModel(CFG, selectModel, pretrained=False, dropOut=0.1)
    efficientNetV2.to(device)
    optimizer2 = torch.optim.Adam(efficientNetV2.parameters(), lr=2e-4)
    print(efficientNetV2)

In [ ]:
if CFG.USE_SWINTransV1:
    swinTranV1 =SWINTransModel(CFG, CFG.modelName3, pretrained=False, dropOut=0.1)
    swinTranV1.to(device)
    optimizer3 = torch.optim.Adam(swinTranV1.parameters(), lr=2e-4)
    print(swinTranV1)

In [ ]:
if CFG.USE_SWINTransV2:
    swinTranV2 =SWINTransModel(CFG, CFG.modelName4, pretrained=False, dropOut=0.1)
    swinTranV2.to(device)
    optimizer4 = torch.optim.Adam(swinTranV2.parameters(), lr=2e-4)
    print(swinTranV2)

In [ ]:
lossFn = nn.BCEWithLogitsLoss()

In [ ]:
# plot  Loss 
def plotLoss(epochList, trainLoss, valLoss, modelName):
    plt.figure(figsize=(6, 4))
    plt.plot(epochList, trainLoss, label='Training  loss')
    plt.plot(epochList, valLoss, label='Validation loss')
    plt.title(f'{modelName} loss (Training/Validation)')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def plotAcc(epochList, trainAcc, valAcc, modelName):
    plt.figure(figsize=(6, 4))
    plt.plot(epochList, trainAcc, label='Training Acc')
    plt.plot(epochList, valAcc, label='Validation Acc')
    plt.title(f'{modelName} Accaury (Training/Validation)')
    plt.xlabel('Epochs')
    plt.ylabel('Accaury')
    plt.legend()
    plt.show()

In [ ]:
def plotpauc(epochList, valPAUC, modelName):
    plt.figure(figsize=(6, 4))
    plt.plot(epochList, valPAUC, label='Validation pAUC')
    plt.title(f'{modelName} Partial AUC (Training/Validation)')
    plt.xlabel('Epochs')
    plt.ylabel('Parical AUC')
    plt.legend()
    plt.show()

# start Training model

In [ ]:
clearTrainHist()

In [ ]:
clearMemory()
memoryInfo()

In [ ]:
# notebook_launcher?

In [ ]:
# Prepare everything with the Accelerator
# There is no order to remember, every object passed in will be returned in that order
# Just remember that anything relating to training and PyTorch should be passed in
# efficientNet1, optimizer1, trainDataLoader, valDataLoader = accelerator.prepare(
#     efficientNet1, optimizer1, trainDataLoader, valDataLoader
#      )

In [ ]:
%%time
if CFG.USE_EFFECTNetv1:
    print("Start Training : efficientNet1")
    print(f"Number of GPU: {torch.cuda.device_count()}")
#     if torch.cuda.device_count() > 1:
#         efficientNet1 = nn.DataParallel(efficientNet1 , device_ids=[0, 1])  # Specify the GPU ids
    trainValLoop(efficientNet1, optimizer1, "efficientNet1", 200) #epochs=CFG.trainEpochs)
#     args = ("fp16", 42, 64)
#     notebook_launcher(trainValLoop(efficientNet1, optimizer1, "efficientNet1", 2) , args, num_processes=2)

In [ ]:
if CFG.USE_EFFECTNetv1:
    plotLoss(epochList, trainLossHist, valLossHist, "efficientNet1")

In [ ]:
if CFG.USE_EFFECTNetv1:
    plotAcc(epochList, trainAccHist, valAccHist, "efficientNet1")

In [ ]:
# valAccHist

In [ ]:
if CFG.USE_EFFECTNetv1:
    plotpauc(epochList, paucHist, "efficientNet1")

In [ ]:
# len(val.target.values)

In [ ]:
clearTrainHist()
clearMemory()

In [ ]:
%%time
if CFG.USE_EFFECTNetv2:
    print("Start Training : efficientNet2")
    print(f"Number of GPU: {torch.cuda.device_count()}")
    trainValLoop(efficientNetV2, optimizer2, "efficientNet2", 200) #epochs=CFG.trainEpochs)

In [ ]:
if CFG.USE_EFFECTNetv2:
    plotLoss(epochList, trainLossHist, valLossHist, "efficientNet2")

In [ ]:
if CFG.USE_EFFECTNetv2:
    plotAcc(epochList, trainAccHist, valAccHist, "efficientNet2")

In [ ]:
if CFG.USE_EFFECTNetv2:
    plotpauc(epochList, paucHist, "efficientNet2")

In [ ]:
clearTrainHist()
clearMemory()

In [ ]:
%%time
if CFG.USE_SWINTransV1:
    print("Start Training : SWIN TransV1")
    print(f"Number of GPU: {torch.cuda.device_count()}")
    trainValLoop(swinTranV1, optimizer3, "swinTransV1", 30) #epochs=CFG.trainEpochs)

In [ ]:
if CFG.USE_SWINTransV1:
    plotLoss(epochList, trainLossHist, valLossHist, "swinTransV1")

In [ ]:
if CFG.USE_SWINTransV1:
    plotAcc(epochList, trainAccHist, valAccHist, "swinTransV1")

In [ ]:
if CFG.USE_SWINTransV1:
    plotpauc(epochList, paucHist, "swinTransV1")

In [ ]:
clearTrainHist()
clearMemory()

In [ ]:
%%time
if CFG.USE_SWINTransV2:
    print("Start Training : SWIN TransV2")
    print(f"Number of GPU: {torch.cuda.device_count()}")
    trainValLoop(swinTranV2, optimizer4, "swinTransV2", 30) #epochs=CFG.trainEpochs)

In [ ]:
if CFG.USE_SWINTransV2:
    plotLoss(epochList, trainLossHist, valLossHist, "swinTransV2")

In [ ]:
if CFG.USE_SWINTransV2:
    plotAcc(epochList, trainAccHist, valAccHist, "swinTransV2")

In [ ]:
if CFG.USE_SWINTransV2:
    plotpauc(epochList, paucHist, "swinTransV2")

In [ ]:
clearTrainHist()
clearMemory()

In [ ]:
def findBestModel():
    bestModel= ""
    bestAcc =0
    bestScore = 0
    bestLoss = np.inf   # init to infinity
    for k, v in modelHistory.items():
        modelAcc = v["best_acc"]
        modelLoss = v["best_loss"]
        modelScore = v["best_score"]
        print("Model: ", k)
        print("Model Best Acc: ", modelAcc)
        print("Model Best Loss: ", modelLoss)
        print("Mobel Best Score: ", modelScore)
        if bestScore < modelScore:
             # update best model
                bestModel = k
                bestAcc = modelAcc
                bestLoss = modelLoss
                bestScore = modelScore
                print("beatModel: ", bestModel )
#         if bestLoss > modelLoss:
#             if bestAcc <  modelAcc:
#                 # update best model
#                 bestModel = k
#                 bestAcc = modelAcc
#                 bestLoss = modelLoss
        
#             print("beatModel: ", bestModel )
        
    if bestModel == "efficientNet1":
        print("Final Model: efficientNet1")
        return efficientNet1     
    elif bestModel == "efficientNet2":
        print("Final Model: efficientNet2")
        return efficientNetV2
    elif bestModel == "swinTransV1":
        print("Final Model: swinTransV1")
        return swinTranV1
    else:
        return None
            
        
        
    
    

In [ ]:
finalModel =  findBestModel()

# Prepare Submit

In [ ]:
submitPred = inferFunc(finalModel, testDataLoader)
submitPred

In [ ]:
submission

In [ ]:
submission["target"]=submitPred

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)